In [25]:
import re

def addxlation(workn):
    
    if(workn < 10):
        workname = 'tlg00' + str(workn)
    else:
        workname = 'tlg0' + str(workn)
    
    inpfname = 'corr.tlg0018.'  + workname + '.yonge_eng1.xml'
    
    f = open(inpfname)
    inimprint = 0
    curdate = ''
    curpublisher = ''
    curplace = ''
    curauthor = 'Philo Judaeus'
    curtranslator = 'Charles Duke Yonge'
    curhead = ''
    for l in f:
        l = re.sub('</*choice>','',l)
        l = re.sub('<sic>[^<]+</sic>','',l)
        l = re.sub('"en"','"eng"',l)
        l = re.sub('"la"','"lat"',l)
        if(re.search('<body>',l)):
            curhead = ''
        if(re.search('<title',l)):
            l = re.sub('<title[^>]+>','',l)
            #print('now',l)
        m = re.search('<head>([^<]+)',l)

        
        if(m and not curhead):
            curhead = m[1]
            curhead = re.sub('\.$','',curhead)
            curhead = re.sub('THE FIRST BOOK OF THE ','',curhead)
        if(re.search('<imprint>',l)):
            inimprint = 1
        if(re.search('</imprint>',l)):
            inimprint = 0
        if(not inimprint):
            continue
        m = re.search('<date>([^<]+)',l)
        if(m):
            curdate = m[1]
            
        m = re.search('<publisher>([^<]+)',l)
        if(m):
            curpublisher = m[1]
            
        m = re.search('<pubPlace>([^<]+)',l)
        if(m):
            curplace = m[1]
            
        m = re.search('<biblscope unit="volume">([^<]+)',l)
        if(m):
            curvolume = m[1]

    
        
    
    f.close()
    
    #outfile = '/Users/gcrane/github/first1kwork/data/tlg0018/' + workname + '/__cts__.xml'
    #outf = open(outfile,'w')
    #metafile = '/Users/gcrane/github/First1KGreek/data/tlg0018/' + workname + '/__cts__.xml'
    #description = '<ti:description>'+curauthor + ', "'+curhead+'," translator: '+curtranslator+ ' ('+curpublisher+', '+curplace+', '+curdate+')' + '</ti:description>'
    #mf = open(metafile)
    #for l in mf:
     #   l = re.sub('\s+$','',l)
    #    l = re.sub('</*choice>','',l)
     #   l = re.sub('<sic>[^<]+</sic>','',l)
      #  l = re.sub('"en"','"eng"',l)
     #   l = re.sub('"la"','"lat"',l)
        
     #   if(re.search('<ti:label',l)):
     #       curlabel = l
     #   if(re.search('</ti:edition>',l)):
     #       print(l,file=outf)
     #       print(file=outf)
            
     #       print('<ti:translation urn="urn:cts:greekLit:tlg0018.'+workname+'.1st1K-eng1" xml:lang="eng" workUrn="urn:cts:greekLit:tlg0018.'+workname+'">',file=outf)
     #       print(curlabel,file=outf)
      #      print('    '+description,file=outf)
      #      print('</ti:translation>',file=outf)
      #      print(file=outf)    
      #      continue
     #   print(l,file=outf)
    
    #mf.close()
    #outf.close()
    
    inpfname = 'corr.tlg0018.'  + workname + '.yonge_eng1.xml'
    
    hasbooks = 0
    f = open(inpfname)
    for l in f:
        if(re.search('<div[^>]+subtype="book"',l)):
            hasbooks = 1
            break
    f.close()
    f = open(inpfname)
    
    outfname = '/Users/gcrane/github/first1kwork/data/tlg0018/'+workname+'/tlg0018.' + workname + '.1st1K-eng1.xml'
    outf = open(outfname,'w')
    curbook = ''
    cursection = ''
    baseurn = 'urn:cts:greekLit:tlg0018.'+workname+'.1st1K-eng1'
    for l in f:
        l = re.sub('\s+$','',l)
        l = re.sub('<teiHeader>','<teiHeader xml:lang="eng">',l)
        l = re.sub('<title>','<title xml:lang="eng">',l)
        l = re.sub('\s+$','',l)
        l = re.sub('</*choice>','',l)
        l = re.sub('<sic>[^<]+</sic>','',l)
        l = re.sub('<corr>([^<]+)</corr>','\g<1>',l)
        l = re.sub('"en"','"eng"',l)
        l = re.sub('"la"','"lat"',l)
        l = re.sub('George Bell and Sons','Henry G. Bohn',l)
        l = re.sub('<pubPlace>London','<pubPlace>York Street, London, Covent Garden',l)
        l = re.sub('<author>Philo Alexandrinus','<author>Philo Judaeus',l)
        
        m = re.search('<div[^>]+subtype="book"[^>]+n="([0-9]+)"',l)
        if(m):
            curbook = m[1]
            cururn = baseurn
            l = re.sub('(<div[^>]+subtype="book")\s+','\g<1> xml:base="' + cururn + '" ',l)
        
        m = re.search('<div[^>]+subtype="section"[^>]+n="([0-9]+)"',l)
        if(m):
            if(curbook):
                cururn = baseurn + ':' + curbook
            else:
                cururn = baseurn
            l = re.sub('(<div[^>]+subtype="section")\s+','\g<1> xml:base="' + cururn + '" ',l)
        
        if(re.search('subtype="edition" n="tlg0018.',l)):
            l = re.sub('type="textpart" subtype="edition"','type="translation"',l)
            l = re.sub('.yonge">','.1st1K-eng1" xml:lang="eng">',l)
            l = re.sub('n="tlg0018','n="urn:cts:greekLit:tlg0018',l)

        if(re.search('<p>The following text is',l)):
            print(l,file=outf)
            if(hasbooks):
                print('<refsDecl n="CTS">',file=outf)
                print('<cRefPattern n="section" matchPattern="(.+).(.+)" replacementPattern="#xpath(/tei:TEI/tei:text/tei:body/tei:div/tei:div[@n=\'$1\']/tei:div[@n=\'$2\'])"/>',file=outf)
                print('<cRefPattern n="book" matchPattern="(.+)" replacementPattern="#xpath(/tei:TEI/tei:text/tei:body/tei:div/tei:div[@n=\'$1\'])"/>',file=outf)
                print('</refsDecl>',file=outf)
            else:
                print('     <refsDecl n="CTS">',file=outf)
                print('        <cRefPattern n="section" matchPattern="(.+)" replacementPattern="#xpath(/tei:TEI/tei:text/tei:body/tei:div/tei:div[@n=\'$1\'])"/>',file=outf)
                print('      </refsDecl>',file=outf)
            continue
        print(l,file=outf)
    
    f.close()
    outf.close()

    
    print(i)
    

for i in range(1,32):
    addxlation(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
